In [1]:
!git clone https://github.com/karpathy/llama2.c.git

Cloning into 'llama2.c'...
remote: Enumerating objects: 1434, done.
remote: Total 1434 (delta 0), reused 0 (delta 0), pack-reused 1434
Receiving objects: 100% (1434/1434), 1.17 MiB | 12.21 MiB/s, done.
Resolving deltas: 100% (871/871), done.


In [2]:
pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.9 MB/s eta 0:00:00


In [3]:
# runtime 100s
!cd llama2.c && python tinystories.py download

data/TinyStories_all_data.tar.gz: 100% 1.50G/1.50G [00:21<00:00, 75.8MiB/s]
Unpacking data/TinyStories_all_data.tar.gz...
Download done.
Number of shards: 50
Example story:
{'story': '\n\nLily and Ben are friends. They like to play in the park. One day, they see a big tree with a swing. Lily wants to try the swing. She runs to the tree and climbs on the swing.\n"Push me, Ben!" she says. Ben pushes her gently. Lily feels happy. She swings higher and higher. She laughs and shouts.\nBen watches Lily. He thinks she is cute. He wants to swing too. He waits for Lily to stop. But Lily does not stop. She swings faster and faster. She is having too much fun.\n"Can I swing too, Lily?" Ben asks. Lily does not hear him. She is too busy swinging. Ben feels sad. He walks away.\nLily swings so high that she loses her grip. She falls off the swing. She lands on the ground. She hurts her foot. She cries.\n"Ow, ow, ow!" she says. She looks for Ben. She wants him to help her. But Ben is not there. He is 

In [4]:
# !cd llama2.c && python tinystories.py train_vocab --vocab_size=4096

!cd ./llama2.c/data && wget https://huggingface.co/datasets/enio/TinyStories/resolve/main/tok4096/tok4096.vocab
!cd ./llama2.c/data && wget https://huggingface.co/datasets/enio/TinyStories/resolve/main/tok4096/tok4096.model

# !cd llama2.c && python tinystories.py pretokenize --vocab_size=4096
!cd ./llama2.c/data && mkdir tok4096
!cd ./llama2.c/data/tok4096 && wget https://huggingface.co/datasets/enio/TinyStories/resolve/main/tok4096/TinyStories_tok4096.tar.gz
!cd ./llama2.c/data/tok4096 && tar -xf TinyStories_tok4096.tar.gz

--2023-12-12 11:46:54--  https://huggingface.co/datasets/enio/TinyStories/resolve/main/tok4096/tok4096.vocab
Resolving huggingface.co (huggingface.co)... 65.8.178.118, 65.8.178.27, 65.8.178.93, ...
Connecting to huggingface.co (huggingface.co)|65.8.178.118|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53534 (52K) [text/plain]
Saving to: ‘tok4096.vocab’

tok4096.vocab       100%[===================>]  52.28K  --.-KB/s    in 0.03s   

2023-12-12 11:46:54 (1.88 MB/s) - ‘tok4096.vocab’ saved [53534/53534]

--2023-12-12 11:46:54--  https://huggingface.co/datasets/enio/TinyStories/resolve/main/tok4096/tok4096.model
Resolving huggingface.co (huggingface.co)... 65.8.178.118, 65.8.178.27, 65.8.178.93, ...
Connecting to huggingface.co (huggingface.co)|65.8.178.118|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/65/c8/65c8615bfd6b6d94c8e62381ee6653578b439f71ff04b86aaefa9828dec8bb70/645406d6a15820e

In [5]:
!cd llama2.c && python tokenizer.py --tokenizer-model=data/tok4096.model

In [6]:
# change bfloat16 to float16 on line 72
!sed -i "72s/.*/dtype = 'float16'  # float32|bfloat16|float16/" ./llama2.c/train.py

In [19]:
# runtime ~ 2 mins
!cd llama2.c && python train.py --vocab_source=custom --vocab_size=4096 --compile=False \
  --dim=128 --n_layers=5 --n_heads=8 --n_kv_heads=4 --batch_size=32 \
  --always_save_checkpoint=True --eval_interval=500 --max_iters=1001 --init_from='resume'

Overriding: vocab_source = custom
Overriding: vocab_size = 4096
Overriding: compile = False
Overriding: dim = 128
Overriding: n_layers = 5
Overriding: n_heads = 8
Overriding: n_kv_heads = 4
Overriding: batch_size = 32
Overriding: always_save_checkpoint = True
Overriding: eval_interval = 500
Overriding: max_iters = 1001
Overriding: init_from = resume
tokens per iteration will be: 32,768
breaks down as: 4 grad accum steps * 1 processes * 32 batch size * 256 max seq len
Resuming training from out
num decayed parameter tensors: 36, with 1,445,888 parameters
num non-decayed parameter tensors: 11, with 1,408 parameters
using fused AdamW: True
Created a PretokDataset with rng seed 42
100 | loss 7.6606 | lr 5.000000e-05 | 741.51ms | mfu -100.00%
101 | loss 7.6589 | lr 5.050000e-05 | 148.91ms | mfu -100.00%
102 | loss 7.6510 | lr 5.100000e-05 | 127.00ms | mfu -100.00%
103 | loss 7.6378 | lr 5.150000e-05 | 125.73ms | mfu -100.00%
104 | loss 7.6287 | lr 5.200000e-05 | 126.58ms | mfu -100.00%
105 

In [8]:
!cd llama2.c && gcc -O3 -o run run.c -lm

In [20]:
!cd ./llama2.c && ./run out/model.bin -z data/tok4096.bin -t 0.8 -n 256 -i "Once upon a time "

Once upon a time 
When they went to the garden, they went to the park to play. They saw a big, shiny hill and he wanted to play. They were very happy.
Suddenly, the bird found a soft box. It looked around and saw a big tree. The bird said, "I will help you open the window." 
He said, "I don't want to play with you!" But the bird was too busy. The bird was not sad anymore. The bird said, "I have a game for you. We can see the bird." 
So, the bird saw the bird. The bird said, "Wow! I'm a magic bird!" The bird felt happy and thanked the bird. The bird and the little bird were very sad. The bird and the bird became good friends and had a fun time in the tree.
achieved tok/s: 644.366197
